## 上一个代码的输出

In [1]:
sqls = [{'question_type': 'check_disease', 'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]

In [2]:
from py2neo import Graph

## 连接图数据库

In [3]:
g = Graph(auth=("neo4j", "neo4jneo4j"))

In [4]:
num_limit = 20

In [5]:
final_answers = []

In [6]:
sqls

[{'question_type': 'check_disease',
  'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]

In [7]:
'''根据对应的qustion_type，调用相应的回复模板'''
def answer_prettify(question_type, answers):
    final_answer = []
    if not answers:
        return ''
    if question_type == 'disease_symptom':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'symptom_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '症状{0}可能染上的疾病有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cause':
        desc = [i['m.cause'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可能的成因有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_prevent':
        desc = [i['m.prevent'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的预防措施包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_lasttime':
        desc = [i['m.cure_lasttime'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治疗可能持续的周期为：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureway':
        desc = [';'.join(i['m.cure_way']) for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可以尝试如下治疗：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureprob':
        desc = [i['m.cured_prob'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治愈的概率为（仅供参考）：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_easyget':
        desc = [i['m.easy_get'] for i in answers]
        subject = answers[0]['m.name']

        final_answer = '{0}的易感人群包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_desc':
        desc = [i['m.desc'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0},熟悉一下：{1}'.format(subject,  '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_acompany':
        desc1 = [i['n.name'] for i in answers]
        desc2 = [i['m.name'] for i in answers]
        subject = answers[0]['m.name']
        desc = [i for i in desc1 + desc2 if i != subject]
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_not_food':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}忌食的食物包括有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_do_food':
        do_desc = [i['n.name'] for i in answers if i['r.name'] == '宜吃']
        recommand_desc = [i['n.name'] for i in answers if i['r.name'] == '推荐食谱']
        subject = answers[0]['m.name']
        final_answer = '{0}宜食的食物包括有：{1}\n推荐食谱包括有：{2}'.format(subject, ';'.join(list(set(do_desc))[:num_limit]), ';'.join(list(set(recommand_desc))[:self.num_limit]))

    elif question_type == 'food_not_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人最好不要吃{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'food_do_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人建议多试试{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'disease_drug':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常的使用的药品包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'drug_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '{0}主治的疾病有{1},可以试试'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_check':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常可以通过以下方式检查出来：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'check_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '通常可以通过{0}检查出来的疾病有{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    return final_answer

In [8]:
for sql_ in sqls:
    question_type = sql_['question_type']
    queries = sql_['sql']
    answers = []
    
    
    for query in queries: # 运行每一条cypher查询语句
        ress = g.run(query).data()
        answers += ress
    final_answer = answer_prettify(question_type, answers)
    if final_answer:
        final_answers.append(final_answer)

In [9]:
ress

[{'m.name': '限制型心肌病', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '病毒性肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '特发性大网膜节段性梗死', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '特发性迟发性免疫球蛋白缺乏症', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '糖原贮积病Ⅴ型', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '糖原贮积病Ⅰ型', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '肠胃炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '慢性肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '炭疽', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '缺血性肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '溃疡性肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '慢性结肠炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '胎粪性便秘', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '扁桃体炎', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '胎粪吸入综合征', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '弹性假黄瘤', 'r.name': '诊断检查', 'n.name': '血常规'},
 {'m.name': '唇疱疹', 

In [10]:
final_answer

'通常可以通过血常规检查出来的疾病有反射性躯体神经病；胃切除后胆汁返流性胃炎；嘧啶5′-核苷酸缺乏症；特发性急性小管间质性肾炎；四边孔综合征；带状疱疹；气管肿瘤；腹股沟直疝（疝气）；神经系统先天性疾病；眩晕症；热带性肺嗜酸粒细胞浸润症；胃溃疡性穿孔；围绝经期前后的外阴白色病变；外阴毛囊炎；溶血性贫血；输血后肝炎；包皮龟头炎；产后三急；小儿肺泡性蛋白沉积症；小提琴手颈部皮炎'